# Behind the Pipeline

In [1]:
from transformers import pipeline

In [6]:
input_list =[
    'when you play game of thrones either you win or you die. There is no middle ground',
    'winter is coming'
]

In [7]:
classifier = pipeline('sentiment-analysis')
classifier(input_list)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9855173230171204},
 {'label': 'POSITIVE', 'score': 0.9872574210166931}]

In [8]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

## Tokenizer
Takes Raw Text as input and converts it into input IDs

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
input = tokenizer(input_list, padding = True, truncation = True, return_tensors = 'tf')
print(input)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'input_ids': <tf.Tensor: shape=(2, 21), dtype=int32, numpy=
array([[ 101, 2043, 2017, 2377, 2208, 1997, 6106, 2015, 2593, 2017, 2663,
        2030, 2017, 3280, 1012, 2045, 2003, 2053, 2690, 2598,  102],
       [ 101, 3467, 2003, 2746,  102,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 21), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>}


# Model
Takes Input IDs and gives Logits as Output

In [14]:
from transformers import TFAutoModel
# This architecture contains only the base Transformer module:
# given some inputs, it outputs what we’ll call hidden states, also known as features.
# For each model input, we’ll retrieve a high-dimensional vector representing the contextual understanding of that input by the Transformer model

model = TFAutoModel.from_pretrained(checkpoint)
output = model(input)
print(output.last_hidden_state.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertMod

(2, 21, 768)


high-dimension-vector = (batch_size, sequence_length, hidden_size)

In [16]:
# For Classification task
from transformers import TFAutoModelForSequenceClassification

classifier_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
classifier_output = classifier_model(input)
print(classifier_output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.2781575, -1.9420531],
       [-2.181795 ,  2.1681824]], dtype=float32)>, hidden_states=None, attentions=None)


# Postprocessing Output

Above results dont make any sense. Needs postprocessing. Since It is classifiaction task, Lets go with softmax


In [18]:
import tensorflow as tf

preds = tf.math.softmax(classifier_output.logits)
print(preds)

tf.Tensor(
[[0.9855173  0.01448272]
 [0.01274263 0.9872573 ]], shape=(2, 2), dtype=float32)


In [20]:
import numpy as np
preds_index = np.argmax(preds, axis =1)
print(preds_index)

[0 1]


In [21]:
classifier_model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [23]:
result = [classifier_model.config.id2label[key] for key in preds_index]
print(result)

['NEGATIVE', 'POSITIVE']
